In [1]:
import os                                     
%pwd

'e:\\2025\\Pojects\\Chest-Cancer-Classification-using-MLflow-DVC\\notebook'

In [2]:
os.chdir('../')

In [3]:
%pwd

'e:\\2025\\Pojects\\Chest-Cancer-Classification-using-MLflow-DVC'

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
MLFLOW_TRACKING_USERNAME = os.getenv('MLFLOW_TRACKING_URI')
MLFLOW_TRACKING_PASSWORD = os.getenv('MLFLOW_TRACKING_PASSWORD')

In [7]:
print(MLFLOW_TRACKING_URI)

https://dagshub.com/mithilesh1627/Chest-Cancer-Classification-using-MLflow-DVC.mlflow


In [8]:
import tensorflow as tf 

In [9]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib  import Path

@dataclass(frozen=True)
class EvalutionConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict 
    mlflow_uri : str
    params_image_size : list
    params_batch_size : int 

In [11]:
from CnnClassifier.constants import *
from CnnClassifier.utils.common import read_yaml,create_directories,save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evalution_config(self) -> EvalutionConfig:
        eval_config = EvalutionConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/Data/train",
            mlflow_uri = MLFLOW_TRACKING_URI,
            params_batch_size=self.params.BATCH_SIZE,
            all_params = self.params,
            params_image_size=self.params.IMAGE_SIZE
        )
        return eval_config

In [17]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
class Evaluation:
    def __init__(self, config: EvalutionConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                # mlflow.ṁkeras.log_model(self.model, "model")
                pass
            else:
                mlflow.keras.log_model(self.model, "model")

In [18]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evalution_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-10-31 07:13:36,118 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-10-31 07:13:36,127 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-10-31 07:13:36,132 : INFO : common : created directory at : artifacts]
Found 182 images belonging to 4 classes.
12/12 [==============================] - 72s 6s/step - loss: 16.5553 - accuracy: 0.3571
[2025-10-31 07:14:49,976 : INFO : common : json file saved at: scores.json]
🏃 View run able-bear-337 at: https://dagshub.com/mithilesh1627/Chest-Cancer-Classification-using-MLflow-DVC.mlflow/#/experiments/0/runs/0457087700694557989a19d8251cce69
🧪 View experiment at: https://dagshub.com/mithilesh1627/Chest-Cancer-Classification-using-MLflow-DVC.mlflow/#/experiments/0
